# Exercise 1

# Exercise 2

# Exercise 3. Paillier Voting System Implementation

The Paillier cryptosystem procedures as requested:

* **`genPrivate(sz)`**: Generates a public key $n$ and private components $p, q$ where $n = p \cdot q$ and $p, q$ are primes of size sz.
* **`voteYes(fileName, n)`**: Encrypts a vote of $1$ and appends it to a file.
* **`voteNo(fileName, n)`**: Encrypts a vote of $0$ and appends it to a file.
* **`getResults(fileName, n, phi)`**: Aggregates the encrypted votes and decrypts the total using Euler's totient, `phi`.

In [5]:
import random
import os
import math

def genPrivate(sz):
    primes = []
    while len(primes) < 2:
        p = random.getrandbits(sz)
        p |= (1 << (sz - 1)) | 1
        if p == 2 or p == 3:
            primes.append(p)
            continue
        if p < 2 or p % 2 == 0:
            continue
        d = p - 1
        r = 0
        while d % 2 == 0:
            d //= 2
            r += 1
        is_prime = True
        for _ in range(40):
            a = random.randrange(2, p - 1)
            x = pow(a, d, p)
            if x == 1 or x == p - 1:
                continue
            for _ in range(r - 1):
                x = pow(x, 2, p)
                if x == p - 1:
                    break
            else:
                is_prime = False
                break
        if is_prime:
            if len(primes) == 0 or primes[0] != p:
                primes.append(p)
    n = primes[0] * primes[1]
    return (n, primes[0], primes[1])

def voteYes(fileName, n):
    n_sq = n * n
    while True:
        r = random.randrange(1, n)
        if math.gcd(r, n) == 1:
            break
    c = ((1 + n) * pow(r, n, n_sq)) % n_sq
    with open(fileName, "a") as f:
        f.write(str(c) + "\n")

def voteNo(fileName, n):
    n_sq = n * n
    while True:
        r = random.randrange(1, n)
        if math.gcd(r, n) == 1:
            break
    c = pow(r, n, n_sq)
    with open(fileName, "a") as f:
        f.write(str(c) + "\n")

def getResults(fileName, n, phi):
    if not os.path.exists(fileName):
        return 0
    n_sq = n * n
    c_product = 1
    with open(fileName, "r") as f:
        for line in f:
            if line.strip():
                c_product = (c_product * int(line.strip())) % n_sq
    c_phi = pow(c_product, phi, n_sq)
    L_u = (c_phi - 1) // n
    phi_inv = pow(phi, -1, n)
    result = (L_u * phi_inv) % n
    print(result)
    return result
if __name__ == "__main__":
    print("--- Initializing Election System ---")
    
    sz = 128
    print(f"Generating {sz}-bit public/private key pairs...")
    n, p, q = genPrivate(sz)
    phi = (p - 1) * (q - 1)
    fname = "poll_data.txt"
    
    if os.path.exists(fname): 
        os.remove(fname)

    num_voters = 25
    expected_yes_votes = 0
    print(f"\nSimulating {num_voters} voters...")

    for i in range(1, num_voters + 1):
        choice = random.choice([0, 1])
        
        if choice == 1:
            voteYes(fname, n)
            expected_yes_votes += 1
            # print(f"Voter {i}: YES") 
        else:
            voteNo(fname, n)
            # print(f"Voter {i}: NO") 

    print("-" * 30)
    print(f"Voting complete. Expected 'YES' count: {expected_yes_votes}")
    print("Processing encrypted ballots from file...")
    
    actual_tally = getResults(fname, n, phi)
    
    if actual_tally == expected_yes_votes:
        print("\nSUCCESS: The encrypted tally matches the manual count.")
    else:
        print("\nERROR: Mismatch detected between tally and manual count.")

--- Initializing Election System ---
Generating 128-bit public/private key pairs...

Simulating 25 voters...
------------------------------
Voting complete. Expected 'YES' count: 11
Processing encrypted ballots from file...
11

SUCCESS: The encrypted tally matches the manual count.
